# Bases de Datos II. NoSQL MongoDB 3 (sesión 4)

```
                                                                  ,gggggggggggg,    ,ggggggggggg,  
                                                                 dP"""88""""""Y8b, dP"""88""""""Y8,
                                                                 Yb,  88       `8b,Yb,  88      `8b
                                                                  `"  88        `8b `"  88      ,8P
                                                                      88         Y8     88aaaad8P"
  ,ggg,,ggg,,ggg,     ,ggggg,    ,ggg,,ggg,     ,gggg,gg    ,ggggg,   88         d8     88""""Y8ba
 ,8" "8P" "8P" "8,   dP"  "Y8ggg,8" "8P" "8,   dP"  "Y8I   dP"  "Y8ggg88        ,8P     88      `8b
 I8   8I   8I   8I  i8'    ,8I  I8   8I   8I  i8'    ,8I  i8'    ,8I  88       ,8P'     88      ,8P
,dP   8I   8I   Yb,,d8,   ,d8' ,dP   8I   Yb,,d8,   ,d8I ,d8,   ,d8'  88______,dP'      88_____,d8'
8P'   8I   8I   `Y8P"Y8888P"   8P'   8I   `Y8P"Y8888P"888P"Y8888P"   888888888P"       88888888P"  
                                                    ,d8I'                                          
                                                  ,dP'8I                                           
                                                 ,8"  8I                                           
                                                 I8   8I                                           
                                                 `8, ,8I                                           
                                                  `Y8P"                             
                                                  
                                                                 
                                             ______
|_  _  _  _  _     _| _     _| _ _|_ _  _     |  |
|_)(_|_> (/__>    (_|(/_   (_|(_| |_(_)_>    _|__|_
```


![MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB_Logo_FullColorBlack_RGB-4td3yuxzjs.png)

Esta hoja es la tercera entrega y última del trabajo con MongoDB. Veremos temas avanzados como cálculos `aggregate()` más complejos, el uso de índices, etc. Al final habrá un conjunto de ejercicios que habrá que resolver y entregar en la tarea abierta para la ocasión.

## Inicio de la hoja


In [1]:
RunningInCOLAB: bool = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

In [2]:
!sudo apt-get update -qq
!sudo apt-get install -y -qq gpg p7zip

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


## Instalación inicial de MongoDB (sólo necesaria en Google Colab)

In [3]:
if RunningInCOLAB:
  !wget -q https://raw.githubusercontent.com/dsevilla/bdge/refs/heads/24-25/addendum/mongo-utils/run-mongo-local.sh
  !bash run-mongo-local.sh

Adding system user `mongodb' (UID 104) ...
Adding new user `mongodb' (UID 104) with group `nogroup' ...
Not creating home directory `/home/mongodb'.
Adding group `mongodb' (GID 107) ...
Done.
Adding user `mongodb' to group `mongodb' ...
Adding user mongodb to group mongodb
Done.
deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/8.0 multiverse
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-

## Descarga de los datos en formato CSV

 - Formato: 7zipped
 - Ficheros:
   - **Comments**.csv
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **Posts**.csv
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **Tags**.csv
    - Id
    - Count
    - ExcerptPostId
    - TagName
    - WikiPostId
   - **Users**.csv
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **Votes**.csv
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

In [4]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001 -O - > es.stackoverflow.csv.7z
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.002 -O - >> es.stackoverflow.csv.7z

--2025-03-06 10:20:36--  https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001 [following]
--2025-03-06 10:20:37--  https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104857600 (100M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 100.00M  73.4MB/s    in 1.4s    

2025-03-06 10:20:46 (73.4 MB/s) - w

In [5]:
!7zr x es.stackoverflow.csv.7z
!rm es.stackoverflow.csv.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 200457538 bytes (192 MiB)

Extracting archive: es.stackoverflow.csv.7z
--
Path = es.stackoverflow.csv.7z
Type = 7z
Physical Size = 200457538
Headers Size = 248
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% - Comments.csv                     2% - Comments.csv                     3% - Comments.csv                     4% - Comments.csv                     6% - Comments.csv                     7% - Comments.csv                     8% - Comments.csv                   

In [6]:
!head Users.csv

Id,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl
-1,"<p>Hola, no soy una persona real.</p><br/><br/><p>¡Soy un proceso que ayuda a mantener el sitio limpio!</p><br/><br/><p>Hago cosas como:</p><br/><br/><ul><br/><li>Dar empujoncitos a preguntas antiguas sin respuesta aproximadamente cada hora, para que atraigan algo de atención.</li><br/><li>Tener la propiedad de las preguntas y respuestas wiki para que nadie se lleve reputación por ellas</li><br/><li>Recibir la propiedad de los votos negativos en las publicaciones de spam o dañinas que son borradas permanentemente</li><br/><li>Tener la propiedad de las ediciones sugeridas por usuarios anónimos</li><br/><li><a href=""http://meta.stackoverflow.com/a/92006"">Quitar preguntas abandonadas</a></li><br/></ul><br/>",-1,2015-10-26T21:36:24.767,Comunidad,22504,2015-10-26T21:36:24.767,en la granja de servidores,1,10211,2516,
1,"<p>Dev #2 who helped create Stack Overflow currently

## Instalación de la librería `pymongo`

In [7]:
%pip install --upgrade pymongo tqdm pandas matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [8]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [9]:
from pymongo import MongoClient

## Conexión a la base de datos

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (`mongo`), o bien a `localhost` si lo estamos haciendo en Colab.

In [10]:
import os
db_hostname: str = "localhost" if RunningInCOLAB else os.getenv('DB_HOSTNAME', "mongo")

In [11]:
from typing import Any

client: MongoClient[dict[str, Any]] = MongoClient(db_hostname,27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [12]:
client.list_database_names()

['admin', 'config', 'local']

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también.

In [13]:
from pymongo.database import Database

db: Database[dict[str,Any]] = client.stackoverflow
# db: Database = client['stackoverflow'] # (equivalente)
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow')

## Importación de los ficheros CSV

Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [14]:
import csv
import sys
from datetime import datetime
from pymongo.collection import Collection
from typing import Any
from collections.abc import Callable, Iterable, Iterator
from tqdm.notebook import tqdm

def batched(iterable: Iterable, n) -> Iterable:
    from itertools import islice
    if n < 1:
        raise ValueError('n must be at least one')
    it: Iterator = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

def csv_to_mongo(file: str, coll: Collection):
    """
    Carga un fichero CSV en Mongo. file especifica el fichero, coll la colección
    dentro de la base de datos, y date_cols las columnas que serán interpretadas
    como fechas.
    """
    # Convertir todos los elementos que se puedan a números
    def to_numeric(d: str) -> str| int | float:
        if len(d) == 0:
            return ''
        if not ((d[0] >= '0' and d[0] <= '9') or d[0] == '-' or d[0] == '+' or d[0]=='.'):
            return d
        try:
            return int(d) if abs(int(d)) <= sys.maxsize else d # Ensure number is inside MongoDB max number range
        except ValueError:
            try:
                return float(d)
            except ValueError:
                return d

    def to_date(d: str) -> datetime | None:
        """To ISO Date. If this cannot be converted, return NULL (None)"""
        try:
            return datetime.strptime(d, "%Y-%m-%dT%H:%M:%S.%f")
        except ValueError:
            return None

    coll.drop()

    with open(file, encoding='utf-8') as f:
        # La llamada csv.reader() crea un iterador sobre un fichero CSV
        reader = csv.reader(f, dialect='excel')

        # Se leen las columnas. Sus nombres se usarán para crear las diferentes columnas en la familia
        columns: list[str] = next(reader)

        # Las columnas que contienen 'Date' se interpretan como fechas
        func_to_cols: list[Callable[[str], Any]] = list(map(lambda c: to_date if 'date' in c.lower() else to_numeric, columns))

        for batch in batched(tqdm(reader, desc='Leyendo e insertando filas...'), 10000):
            docs: list[dict[str, str | datetime | float | int | None]] = []
            for row in batch:
                built_row: list[str | datetime | float | int | None] = [func(e) for (func,e) in zip(func_to_cols, row)]
                docs.append(dict(zip(columns, built_row)))
            coll.insert_many(docs)

        print("¡Hecho!")

In [15]:
csv_to_mongo('Posts.csv',db.posts)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [16]:
csv_to_mongo('Users.csv',db.users)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [17]:
csv_to_mongo('Votes.csv',db.votes)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [18]:
csv_to_mongo('Comments.csv',db.comments)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [19]:
csv_to_mongo('Tags.csv',db.tags)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [20]:
posts: Collection[dict[str,Any]] = db.posts
posts

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow'), 'posts')

In [21]:
posts.count_documents({})

410346

### El API de colección de MongoDB

El API de colección en Python se puede encontrar aquí: https://docs.mongodb.com/drivers/pymongo/. La mayoría de libros y referencias muestran el uso de mongo desde Javascript, ya que el *shell* de MongoDB acepta ese lenguaje. La sintaxis con respecto a Python cambia un poco, y se puede seguir en el enlace anterior. Existe incluso un curso de la MongoDB University que se puede realizar de forma gratuita: https://learn.mongodb.com/learning-paths/using-mongodb-with-python

## Framework de Agregación

Framework de agregación:
- Aquí está la referencia de las diferentes etapas por las que puede pasar un pipeline: https://www.mongodb.com/docs/manual/reference/operator/aggregation-pipeline/.
- Y aquí los distintos operadores que se permiten dentro de las etapas: https://docs.mongodb.com/manual/reference/operator/aggregation/.
- Y aquí incluso un libro completo con usos prácticos de ejecutar agregación: https://www.practical-mongodb-aggregations.com/.

A continuación otro vídeo interseante:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('VSX4a3h4SmQ',width=600)

![](https://miro.medium.com/max/1060/1*2lDBxvZ8Cr3JYkoODTa0lQ.png)

Proyección: el operador `$project` permite filtar qué campos de los documentos queremos usar en la siguientes fases de agregación.

In [22]:
from pymongo.command_cursor import CommandCursor

respuestas: CommandCursor[dict[str,Any]] = db['posts'].aggregate( [
    {'$project' : { 'Id' : True }},
    {'$limit': 20} ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1b1'), 'Id': 1},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b2'), 'Id': 2},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b3'), 'Id': 3},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b4'), 'Id': 4},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b5'), 'Id': 5},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b6'), 'Id': 6},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b7'), 'Id': 7},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b8'), 'Id': 8},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1b9'), 'Id': 9},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1ba'), 'Id': 10},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1bb'), 'Id': 11},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1bc'), 'Id': 12},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1bd'), 'Id': 14},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1be'), 'Id': 15},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1bf'), 'Id': 16},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1c0'), 'Id': 17},
 {'_id': ObjectId('67c97749b8bb3c9524f9d1c1'), 'Id': 18},
 {'_id': ObjectId('67c9

_Lookup_! El operador `$lookup` permite realizar búsquedas en otras colecciones. Podrían interpretarse como un `join` en el modelo relacional.

In [23]:
respuestas: CommandCursor[dict[str,Any]] = posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 150}}},
        {'$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"}
        }
        ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1d5'),
  'Id': 40,
  'AcceptedAnswerId': '',
  'AnswerCount': '',
  'Body': '<p>Asumiendo que se está haciendo un <code>join</code> de columnas sin duplicados, lo cuál es un caso común:</p><br/><ul><br/><li><p>Un <strong>inner join</strong> de A y B entregará el resultado de la intersección de los conjuntos A y B. En otras palabras, la parte <em>interna</em> –intersección– en un diagrama de Venn.</p><br/></li><br/><li><p>Un <strong>full outer join</strong> entre A y B entregará el resultado de la unión de A y B. En otras palabras, la parte <em>externa</em> –unión– en un diagrama de Venn .</p><br/></li><br/></ul><br/><h1>Ejemplos:</h1><br/><p>Supongamos que tenemos dos tablas, con una sola columna cada una y los siguientes datos:</p><br/><pre><code>A    B<br/>-    -<br/>1    3<br/>2    4<br/>3    5<br/>4    6<br/></code></pre><br/><p>Nota que (1,2) solo se encuentran en A, (3,4) son comunes y (5,6) solamente se encuentran en B.</p><br/><h1>Inner J

El `$lookup` genera un _array_ con todos los resultados. El operador `$arrayElementAt` accede al primer elemento.

In [24]:
respuestas: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 150}}},
        {'$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"}
        },
        { '$project' : {
            'Id' : True,
            'Score' : True,
            'username' : {'$arrayElemAt' : ['$owner.DisplayName', 0]},
            'owner.DisplayName' : True
          }}
        ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1d5'),
  'Id': 40,
  'Score': 206,
  'owner': [{'DisplayName': 'jachguate'}],
  'username': 'jachguate'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d255'),
  'Id': 197,
  'Score': 176,
  'owner': [{'DisplayName': 'Peter Olson'}],
  'username': 'Peter Olson'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d259'),
  'Id': 204,
  'Score': 195,
  'owner': [{'DisplayName': 'Konamiman'}],
  'username': 'Konamiman'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a23'),
  'Id': 18232,
  'Score': 232,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a24'),
  'Id': 18233,
  'Score': 219,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c97755b8bb3c9524fad2da'),
  'Id': 87842,
  'Score': 173,
  'owner': [{'DisplayName': 'Alvaro Montoro'}],
  'username': 'Alvaro Montoro'}]

`$unwind` también puede usarse. "Desdobla" cada fila por cada elemento del array. En este caso, como sabemos que el array sólo contiene un elemento, sólo habrá una fila por fila original, pero sin el _array_. Finalmente se puede proyectar el campo que se quiera. Volvemos a proyectar los campos anteriores para comprobar las diferencias.

In [25]:
respuestas: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 150}}},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'}, # Elimina la lista que da project y pondría el post junto cada usuario propietario por separado. Habría tantos posts como propietarios
        { '$project' : {
            'Id' : True,
            'Score' : True,
            'username': '$owner.DisplayName',
            'owner.DisplayName' : True
          }
        }
        ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1d5'),
  'Id': 40,
  'Score': 206,
  'owner': {'DisplayName': 'jachguate'},
  'username': 'jachguate'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d255'),
  'Id': 197,
  'Score': 176,
  'owner': {'DisplayName': 'Peter Olson'},
  'username': 'Peter Olson'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d259'),
  'Id': 204,
  'Score': 195,
  'owner': {'DisplayName': 'Konamiman'},
  'username': 'Konamiman'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a23'),
  'Id': 18232,
  'Score': 232,
  'owner': {'DisplayName': 'Alvaro Montoro'},
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a24'),
  'Id': 18233,
  'Score': 219,
  'owner': {'DisplayName': 'Alvaro Montoro'},
  'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c97755b8bb3c9524fad2da'),
  'Id': 87842,
  'Score': 173,
  'owner': {'DisplayName': 'Alvaro Montoro'},
  'username': 'Alvaro Montoro'}]

Repetimos la consulta anterior, pero proyectando solamente el campo 'username'.

In [26]:
respuestas: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 150}}},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
            'username': '$owner.DisplayName',
          }
        }
        ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1d5'), 'username': 'jachguate'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d255'), 'username': 'Peter Olson'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d259'), 'username': 'Konamiman'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a23'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a24'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c97755b8bb3c9524fad2da'), 'username': 'Alvaro Montoro'}]

Veamos cómo se ha realizado la consulta anterior. Para ello usamos la opción `explain`. En el campo `strategy` que se ha usado `NestedLoopJoin`. Para cada documento en `posts`, busca ''manualmente'' en `users` los documentos que coincidan con `OwnerUserId`.

In [27]:
db.command('aggregate', 'posts', pipeline= [
  { '$match': { 'Score' : {'$gte': 150}}},
  { '$lookup': {
      'from': "users",
      'localField': "OwnerUserId",
      'foreignField': "Id",
      'as': "owner"
    }
  },
  { '$unwind': '$owner'},
  { '$project': {
      'username': '$owner.DisplayName',

     }
  }
  ],
  explain = True
)

{'explainVersion': '1',
 'stages': [{'$cursor': {'queryPlanner': {'namespace': 'stackoverflow.posts',
     'parsedQuery': {'Score': {'$gte': 150}},
     'indexFilterSet': False,
     'queryHash': 'FE018D7F',
     'planCacheShapeHash': 'FE018D7F',
     'planCacheKey': '632B2F53',
     'optimizationTimeMillis': 0,
     'maxIndexedOrSolutionsReached': False,
     'maxIndexedAndSolutionsReached': False,
     'maxScansToExplodeReached': False,
     'prunedSimilarIndexes': False,
     'winningPlan': {'isCached': False,
      'stage': 'PROJECTION_SIMPLE',
      'transformBy': {'OwnerUserId': 1, '_id': 1},
      'inputStage': {'stage': 'COLLSCAN',
       'filter': {'Score': {'$gte': 150}},
       'direction': 'forward'}},
     'rejectedPlans': []}}},
  {'$lookup': {'from': 'users',
    'as': 'owner',
    'localField': 'OwnerUserId',
    'foreignField': 'Id',
    'unwinding': {'preserveNullAndEmptyArrays': False}}},
  {'$project': {'_id': True, 'username': '$owner.DisplayName'}}],
 'queryShapeH

## Definición de índices

Las consultas anteriores no se han podido realizar, ya que el uso de `$lookup` (equivalente a un JOIN de SQL) hace que se tenga que buscar en toda la colección usuarios para encontrar el usuario con `Id` especificado por `OwnerUserId`. Vamos a probarlas sin limitación después de crear un índice.


In [28]:
from pymongo import HASHED

db.users.create_index(['Id', HASHED])

'Id_1_hashed_1'

In [29]:
respuestas: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
  { '$match': { 'Score' : {'$gte': 150}}},
  { '$lookup': {
      'from': "users",
      'localField': "OwnerUserId",
      'foreignField': "Id",
      'as': "owner"
    }
  },
  { '$unwind': '$owner'},
  { '$project' : {
        'username': '$owner.DisplayName'
    }
  }
  ])
list(respuestas)

[{'_id': ObjectId('67c97749b8bb3c9524f9d1d5'), 'username': 'jachguate'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d255'), 'username': 'Peter Olson'},
 {'_id': ObjectId('67c97749b8bb3c9524f9d259'), 'username': 'Konamiman'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a23'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c9774ab8bb3c9524fa0a24'), 'username': 'Alvaro Montoro'},
 {'_id': ObjectId('67c97755b8bb3c9524fad2da'), 'username': 'Alvaro Montoro'}]

Veamos cómo se ha usado el índice en la consulta (no se incluye la consulta entera porque el explain no lo termina de hacer bien si hay muchos pasos). De nuevo usamos la opción `explain`. En el campo `strategy` que se ha usado `IndexedLoopJoin`. En este caso se utiliza el índice creado en `users` para hacer la búsqueda de una forma más rápida y eficiente.

In [30]:
db.command('aggregate', 'posts', pipeline= [
  { '$match': { 'Score' : {'$gte': 150}}},
  { '$lookup': {
      'from': "users",
      'localField': "OwnerUserId",
      'foreignField': "Id",
      'as': "owner"
    }
  }],
  explain = True
)

{'explainVersion': '2',
 'queryPlanner': {'namespace': 'stackoverflow.posts',
  'parsedQuery': {'Score': {'$gte': 150}},
  'indexFilterSet': False,
  'queryHash': '0873BBE5',
  'planCacheShapeHash': '0873BBE5',
  'planCacheKey': 'D1416C01',
  'optimizationTimeMillis': 0,
  'optimizedPipeline': True,
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'prunedSimilarIndexes': False,
  'winningPlan': {'isCached': False,
   'queryPlan': {'stage': 'EQ_LOOKUP',
    'planNodeId': 2,
    'foreignCollection': 'stackoverflow.users',
    'localField': 'OwnerUserId',
    'foreignField': 'Id',
    'asField': 'owner',
    'strategy': 'IndexedLoopJoin',
    'indexName': 'Id_1_hashed_1',
    'indexKeyPattern': {'Id': 1, 'hashed': 1},
    'scanDirection': 'forward',
    'inputStage': {'stage': 'COLLSCAN',
     'planNodeId': 1,
     'filter': {'Score': {'$gte': 150}},
     'direction': 'forward'}},
   'slotBasedPlan': {'slots': '$$RES

Calculemos la media de puntuación por usuario:

In [31]:
media_puntuación_usuarios: CommandCursor[dict[str,Any]] = db.posts.aggregate([
    {
        '$group': {
            '_id': '$OwnerUserId',
            'media_puntuación': { '$avg':  '$Score' }
            }
    },
    { '$limit': 100 }
    ])
list(media_puntuación_usuarios)

[{'_id': 234449, 'media_puntuación': 0.0},
 {'_id': 151829, 'media_puntuación': 0.0},
 {'_id': 155573, 'media_puntuación': 1.0},
 {'_id': 152793, 'media_puntuación': 1.0},
 {'_id': 326335, 'media_puntuación': -1.0},
 {'_id': 34802, 'media_puntuación': 2.0},
 {'_id': 50660, 'media_puntuación': 0.0},
 {'_id': 187124, 'media_puntuación': 0.0},
 {'_id': 286732, 'media_puntuación': 0.0},
 {'_id': 249483, 'media_puntuación': 1.0},
 {'_id': 238023, 'media_puntuación': -1.0},
 {'_id': 224843, 'media_puntuación': -1.0},
 {'_id': 334067, 'media_puntuación': 0.0},
 {'_id': 163609, 'media_puntuación': 2.0},
 {'_id': 290377, 'media_puntuación': 0.0},
 {'_id': 6238, 'media_puntuación': 1.1666666666666667},
 {'_id': 345471, 'media_puntuación': 0.0},
 {'_id': 95259, 'media_puntuación': 1.0},
 {'_id': 299546, 'media_puntuación': 0.0},
 {'_id': 205999, 'media_puntuación': 0.0},
 {'_id': 183696, 'media_puntuación': 0.0},
 {'_id': 147149, 'media_puntuación': 0.75},
 {'_id': 277991, 'media_puntuación': 0.0

Calculemos el máximo y mínimo de puntuación de cada usuario:

In [32]:
max_min_puntuación_usuarios: CommandCursor[dict[str,Any]] = db.posts.aggregate([
    {
        '$group': {
            '_id': '$OwnerUserId',
            'media_puntuación': { '$avg':  '$Score' },
            'min': { '$min':  '$Score' },
            'max': { '$max' : "$Score"}
        }
    },
    { '$limit': 100 }
    ])
list(max_min_puntuación_usuarios)

[{'_id': 234449, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 151829, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 155573, 'media_puntuación': 1.0, 'min': 0, 'max': 3},
 {'_id': 152793, 'media_puntuación': 1.0, 'min': 1, 'max': 1},
 {'_id': 326335, 'media_puntuación': -1.0, 'min': -1, 'max': -1},
 {'_id': 34802, 'media_puntuación': 2.0, 'min': 2, 'max': 2},
 {'_id': 50660, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 187124, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 286732, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 249483, 'media_puntuación': 1.0, 'min': 1, 'max': 1},
 {'_id': 238023, 'media_puntuación': -1.0, 'min': -1, 'max': -1},
 {'_id': 224843, 'media_puntuación': -1.0, 'min': -1, 'max': -1},
 {'_id': 334067, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 163609, 'media_puntuación': 2.0, 'min': 2, 'max': 2},
 {'_id': 290377, 'media_puntuación': 0.0, 'min': 0, 'max': 0},
 {'_id': 6238, 'media_puntuación': 1.16666666666

Pasemos ahora a trabajar con la colección Votes. Antes de empezar, veamos las primeras líneas del fichero `Votes.csv`:

In [33]:
!head Votes.csv

Id,BountyAmount,CreationDate,PostId,UserId,VoteTypeId
1,,2015-10-29T00:00:00.000,1,,2
2,,2015-10-29T00:00:00.000,2,,2
3,,2015-10-30T00:00:00.000,3,,2
4,,2015-10-30T00:00:00.000,5,,2
5,,2015-10-30T00:00:00.000,7,,2
7,,2015-10-30T00:00:00.000,3,,16
8,,2015-10-30T00:00:00.000,4,,2
9,,2015-10-30T00:00:00.000,1,,2
10,,2015-10-30T00:00:00.000,1,,16


Podemos comprobar que todos los posts tienen los votos correctos contando los votos positivos (`VoteTypeId=2`) y negativos (`VoteTypeId=3`).

Primero, calcular el resultado de cada voto. Nótese cómo haciendo uso de `$cond` se puede aplicar una condición `if` para sumar o restar dependiendo del tipo del voto.

Después lo comprobaremos con los existentes en Posts.

In [34]:
score_by_post: CommandCursor[dict[str,Any]] = db.votes.aggregate([
    { "$match" : {"$or" : [ {"VoteTypeId" : 2},
                            {"VoteTypeId" : 3}]
                  }
    },
    {
        '$group': {
            '_id': '$PostId',
            'score': { '$sum': { "$cond": {
                                            "if" : { "$eq" : [ "$VoteTypeId", 2 ]},
                                            "then" : 1,
                                            "else" : -1
                                          }} }
        }
    },
    { '$limit' : 20}
    ])
list(score_by_post)

[{'_id': 319993, 'score': 1},
 {'_id': 280924, 'score': 1},
 {'_id': 98307, 'score': -1},
 {'_id': 4417, 'score': 1},
 {'_id': 98557, 'score': -3},
 {'_id': 246211, 'score': 9},
 {'_id': 277252, 'score': 1},
 {'_id': 233177, 'score': 3},
 {'_id': 399420, 'score': -2},
 {'_id': 422210, 'score': -3},
 {'_id': 440038, 'score': 1},
 {'_id': 373815, 'score': -1},
 {'_id': 39755, 'score': 4},
 {'_id': 567759, 'score': 1},
 {'_id': 229997, 'score': 0},
 {'_id': 210550, 'score': -1},
 {'_id': 183275, 'score': -1},
 {'_id': 539698, 'score': -2},
 {'_id': 552638, 'score': 1},
 {'_id': 19598, 'score': 1}]

Y después comparado con lo que hay en los Posts. La consulta, como se puede ver, sacará sólo los elementos en donde los votos en `Votes` no coincide con `Posts` (debe ser vacío). Nótese el uso de la nueva construcción `$expr`, que permite añadir expresiones dentro de la consulta.

**CUIDADO** Si no se define el índice correcto, la consulta tarda mucho. ¿Cuál?

In [35]:
# Creación del índice secreto... ¿cuál?
import zlib
secret_code = b'x\x9cKI\xd2+\xc8/.)\xd6K.JM,I\x8d\xcf\xccKI\xad\xd0\x88V\xf7LQ\xd7Q\xf0p\x0c\xf6pu\x89\xd5\x04\x00\xfe`\x0cD'
eval(zlib.decompress(secret_code).decode())

'Id_1_hashed_1'

In [36]:
score_by_post_comp: CommandCursor[dict[str,Any]] = db.votes.aggregate([
    { "$match" : {"$or" : [ {"VoteTypeId" : 2},
                            {"VoteTypeId" : 3}]
                  }
    },
    {
        '$group': {
            '_id': '$PostId',
            'score': { '$sum': { "$cond" : {
                                            "if" : { "$eq" : [ "$VoteTypeId", 2 ]},
                                            "then" : 1,
                                            "else" : -1
                                           }}}
        }
    },
    {
        "$lookup": {
            'from': "posts",
            'localField': "_id",
            'foreignField': "Id",
            'as': "post"
        }
    },
    { "$unwind" : "$post"},
    {
        "$match" : {'$expr' : {'$ne' : [ '$score', '$post.Score']}}
    }
])
list(score_by_post_comp)

[]

### Ejemplo de realización de una consulta de reciprocidad

Como ejemplo de consulta compleja con el Framework de Agregación, adjunto una posible solución a la consulta siguiente: ¿Hay usuarios que hacen preguntas y son respondidos por otros usuarios que a su vez preguntaron y fueron respondidos por los primeros? (Es decir, ¿hay reciprocidad en las interacciones en Stacokverflow español?)

In [37]:
RQ4: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
        { "$match" : {"PostTypeId": 2}},
        { '$lookup': {
            'from': "posts",
            'localField': "ParentId",
            'foreignField': "Id",
            'as': "question"
          }
        },

        { '$unwind' : '$question' },

        { '$project' : {
            'OwnerUserId': True,
            'OP' : '$question.OwnerUserId'
          }
        },
        { '$group' : {'_id' : {'min' : { '$min' : ['$OwnerUserId' , '$OP'] },
                               'max' : { '$max' : ['$OwnerUserId' , '$OP'] }},
                      'pairs' : {'$addToSet' : { '0q': '$OP', '1a': '$OwnerUserId'}}
          }
        },
        { '$project': {
            'pairs' : True,
            'npairs' : { '$size' : '$pairs'}
          }
        },
        { '$match' : { 'npairs' : { '$eq' : 2}} }
    ])
RQ4 = list(RQ4)
RQ4

[{'_id': {'min': 44942, 'max': 44946},
  'pairs': [{'0q': 44946, '1a': 44942}, {'0q': 44942, '1a': 44946}],
  'npairs': 2},
 {'_id': {'min': 28050, 'max': 30751},
  'pairs': [{'0q': 28050, '1a': 30751}, {'0q': 30751, '1a': 28050}],
  'npairs': 2},
 {'_id': {'min': 16349, 'max': ''},
  'pairs': [{'0q': '', '1a': 16349}, {'0q': 16349, '1a': ''}],
  'npairs': 2},
 {'_id': {'min': 19533, 'max': 38304},
  'pairs': [{'0q': 38304, '1a': 19533}, {'0q': 19533, '1a': 38304}],
  'npairs': 2},
 {'_id': {'min': 250, 'max': 2429},
  'pairs': [{'0q': 250, '1a': 2429}, {'0q': 2429, '1a': 250}],
  'npairs': 2},
 {'_id': {'min': 228290, 'max': ''},
  'pairs': [{'0q': '', '1a': 228290}, {'0q': 228290, '1a': ''}],
  'npairs': 2},
 {'_id': {'min': 106915, 'max': 134369},
  'pairs': [{'0q': 106915, '1a': 134369}, {'0q': 134369, '1a': 106915}],
  'npairs': 2},
 {'_id': {'min': 48306, 'max': 55380},
  'pairs': [{'0q': 55380, '1a': 48306}, {'0q': 48306, '1a': 55380}],
  'npairs': 2},
 {'_id': {'min': 6564, 'ma

En el caso de que queramos tener como referencia las preguntas y respuestas a las que se refiere la conversación, se puede añadir un campo más que guarde todas las preguntas junto con sus respuestas consideradas:

In [38]:
RQ4: CommandCursor[dict[str,Any]] = db.posts.aggregate( [
        { '$match': { 'PostTypeId' : 2}},
        { '$lookup': {
            'from': "posts",
            'localField': "ParentId",
            'foreignField': "Id",
            'as': "question"
          }
        },

        { '$unwind' : '$question' },

        { '$project' : {
            'OwnerUserId': True,
            'QId' : '$question.Id',
            'AId' : '$Id',
            'OP' : '$question.OwnerUserId'
          }
        },
        { '$group' : {'_id' : {'min' : { '$min' : ['$OwnerUserId' , '$OP'] },
                               'max' : { '$max' : ['$OwnerUserId' , '$OP'] }},
                      'pairs' : {'$addToSet' : { '0q':'$OP', '1a': '$OwnerUserId'}},
                      'considered_pairs' : { '$push' : {'QId' : '$QId', 'AId' : '$AId'}}
          }
        },
        { '$project': {
            'pairs' : True,
            'npairs' : { '$size' : '$pairs'},
            'considered_pairs' : True
          }
        },
        { '$match' : { 'npairs' : { '$eq' : 2}} }
    ])
RQ4 = list(RQ4)
RQ4

[{'_id': {'min': 21, 'max': 1184},
  'pairs': [{'0q': 1184, '1a': 21}, {'0q': 21, '1a': 1184}],
  'considered_pairs': [{'QId': 2615, 'AId': 2619},
   {'QId': 36, 'AId': 47441},
   {'QId': 4630, 'AId': 446773}],
  'npairs': 2},
 {'_id': {'min': 22, 'max': 288},
  'pairs': [{'0q': 288, '1a': 22}, {'0q': 22, '1a': 288}],
  'considered_pairs': [{'QId': 565, 'AId': 604}, {'QId': 731, 'AId': 734}],
  'npairs': 2},
 {'_id': {'min': 22, 'max': 342},
  'pairs': [{'0q': 342, '1a': 22}, {'0q': 22, '1a': 342}],
  'considered_pairs': [{'QId': 618, 'AId': 623}, {'QId': 552, 'AId': 832}],
  'npairs': 2},
 {'_id': {'min': 22, 'max': 729},
  'pairs': [{'0q': 22, '1a': 729}, {'0q': 729, '1a': 22}],
  'considered_pairs': [{'QId': 2454, 'AId': 2459}, {'QId': 1919, 'AId': 2734}],
  'npairs': 2},
 {'_id': {'min': 23, 'max': 25},
  'pairs': [{'0q': 25, '1a': 23}, {'0q': 23, '1a': 25}],
  'considered_pairs': [{'QId': 48, 'AId': 52},
   {'QId': 803, 'AId': 805},
   {'QId': 1015, 'AId': 1095}],
  'npairs': 2},


Comprobemos ahora cuatro entradas que están relacionadas dos a dos.

In [39]:
(db.posts.find_one({'Id': 238}), db.posts.find_one({'Id': 243}),
db.posts.find_one({'Id': 222}), db.posts.find_one({'Id': 223}))

({'_id': ObjectId('67c97749b8bb3c9524f9d279'),
  'Id': 238,
  'AcceptedAnswerId': 243,
  'AnswerCount': 1,
  'Body': "<p>Estoy observando un mensaje de alerta (no es un error) <code>InsecurePlatformWarning</code> al ejecutar el comando <code>pip</code> en Linux:</p><br/><br/><pre><code>$ pip search reportlab<br/>/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:90:<br/>         InsecurePlatformWarning: A true SSLContext object is not available.<br/>         This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail.<br/>         For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.<br/>  InsecurePlatformWarning<br/>reportlab           - The ReportLab Toolkit<br/></code></pre><br/><br/><p>Tengo instalada la última versión de <code>pip</code>:</p><br/><br/><pre><code>$ pip show pip<br/>---<br/>Metadata-Version: 2.0<br/>Name: pip<br/>Version: 7.1.2<br/>S

### Ejemplo de consulta: Tiempo medio desde que se hace una pregunta hasta que se le da la primera respuesta

Veamos cómo calcular el tiempo medio desde que se hace una pregunta hasta que se le da la primera respuesta. No se considerarán pues las preguntas que no tienen respuesta, lo cual es razonable.

Esto sólo calcula el tiempo mínimo de cada pregunta a su respuesta. Después se añadirá el código que calcula la media final:

In [40]:
min_answer_time: CommandCursor[dict[str,Any]] = db.posts.aggregate([
    { "$match" : {"PostTypeId" : 2} },
    { '$group' : {
        '_id' : '$ParentId',
        # 'answers' : { '$push' : {'Id' : "$Id", 'CreationDate' : "$CreationDate"}},
        'min' : {'$min' : "$CreationDate"}
      }
    },
    { "$lookup" : {
        'from': "posts",
        'localField': "_id",
        'foreignField': "Id",
        'as': "post"}
    },
    { "$unwind" : "$post"},
    { "$project" : {
        "_id" : True,
        "min" : True,
        "diff" : {"$subtract" : ["$min", "$post.CreationDate"]}
      }
    }
])
min_answer_time = list(min_answer_time)
min_answer_time

[{'_id': 129977,
  'min': datetime.datetime(2018, 1, 9, 23, 19, 37, 693000),
  'diff': 1418606},
 {'_id': 265402,
  'min': datetime.datetime(2019, 5, 21, 9, 39, 58, 860000),
  'diff': 1960950},
 {'_id': 100846,
  'min': datetime.datetime(2017, 9, 8, 14, 8, 11, 307000),
  'diff': 134777},
 {'_id': 97091,
  'min': datetime.datetime(2017, 8, 24, 13, 42, 35, 13000),
  'diff': 569856},
 {'_id': 196348,
  'min': datetime.datetime(2018, 9, 16, 0, 11, 33, 887000),
  'diff': 229116760},
 {'_id': 388577,
  'min': datetime.datetime(2020, 9, 9, 22, 24, 42, 730000),
  'diff': 2003913},
 {'_id': 454701,
  'min': datetime.datetime(2021, 5, 25, 2, 34, 48, 680000),
  'diff': 3674970},
 {'_id': 585320,
  'min': datetime.datetime(2023, 3, 2, 10, 49, 34, 770000),
  'diff': 45696697},
 {'_id': 352498,
  'min': datetime.datetime(2020, 5, 5, 16, 21, 46, 563000),
  'diff': 1332260},
 {'_id': 207657,
  'min': datetime.datetime(2018, 10, 25, 13, 0, 27, 987000),
  'diff': 52491247},
 {'_id': 597408,
  'min': dat

La siguiente consulta sí calcula la media:

In [41]:
avg_answer_time: CommandCursor[dict[str,Any]] = db.posts.aggregate([
    { "$match" : {"PostTypeId" : 2} },
    { '$group' : {
        '_id' : '$ParentId',
        # 'answers' : { '$push' : {'Id' : "$Id", 'CreationDate' : "$CreationDate"}},
        'min' : {'$min' : "$CreationDate"}
      }
    },
    { "$lookup" : {
        'from': "posts",
        'localField': "_id",
        'foreignField': "Id",
        'as': "post"}
    },
    { "$unwind" : "$post"},
    { "$project" : {
        "_id" : True,
        "min" : True,
        "diff" : {"$subtract" : ["$min", "$post.CreationDate"]}
      }
    },
    #    { "$sort" : {'_id' : 1} }
    {
      "$group" :  {
        "_id" : None,
        "avg" : { "$avg" : "$diff"}
      }
    }
])
avg_answer_time = list(avg_answer_time)
avg_answer_time

[{'_id': None, 'avg': 943326388.0748069}]

## EJERCICIO 1: Listar aquellos usuarios con una reputación (`Reputation`) superior o igual a 500 y un número de visualizaciones (`Views`) superior a 200. De dichos usuarios solo mostrar los campos:
- `DisplayName`
- `Location`
- `Reputation`
- `Views`

*IMPORTANTE: No mostrar el campo `_id`*

In [46]:
usuarios_importantes: CommandCursor[dict[str,Any]] = db.users.aggregate([
    {"$match": {"$and": [{"Reputation": {"$gte": 500}},
                {"Views": {"$gt": 200}}]}},

    {"$project": {
        "_id": False,
        "DisplayName": True,
        "Location": True,
        "Reputation": True,
        "Views": True
    }}])

usuarios_importantes = list(usuarios_importantes)
usuarios_importantes


[{'DisplayName': 'Carlos Muñoz',
  'Location': 'Lima, Peru',
  'Reputation': 13368,
  'Views': 2231},
 {'DisplayName': 'jachguate',
  'Location': 'Guatemala',
  'Reputation': 30128,
  'Views': 2892},
 {'DisplayName': 'Konamiman',
  'Location': 'Palma de Mallorca, Spain',
  'Reputation': 5278,
  'Views': 787},
 {'DisplayName': 'Angel Angel',
  'Location': '',
  'Reputation': 10093,
  'Views': 1023},
 {'DisplayName': 'rnrneverdies',
  'Location': 'Argentina',
  'Reputation': 16867,
  'Views': 1643},
 {'DisplayName': 'Lamak',
  'Location': 'Chile',
  'Reputation': 8505,
  'Views': 1348},
 {'DisplayName': 'Diego', 'Location': '', 'Reputation': 5693, 'Views': 332},
 {'DisplayName': 'Davlio',
  'Location': 'Lima, Peru',
  'Reputation': 2980,
  'Views': 333},
 {'DisplayName': 'Rubén',
  'Location': 'Monterrey, Mexico',
  'Reputation': 11406,
  'Views': 1886},
 {'DisplayName': 'Shaz',
  'Location': 'Canada / Colombia',
  'Reputation': 29168,
  'Views': 4246},
 {'DisplayName': 'astrojuanlu',
  

## EJERCICIO 2: Listar todos post que son preguntas y sus votos asociados. Para ello, se permite definir los indices que se consideren adecuados.

*Ejemplo*: Suponiendo que el post con `_id` `65e6f1c8d93b356db73e5433` tuviera tres votos asociados, la solución debe devolver una salida de en este formato (nótese que los campos `CreationDate`, `Score` e `_id` se refieren al mismo post):

```json
{'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 2}},
 {'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 2}},
 {'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 16}}
```


## EJERCICIO 3: Listar los nombres de usuarios (campo `DisplayName` de `users`) que tienen más preguntas que respuestas

(Contando los usuarios debe salir exactamente 54483 usuarios).

## EJERCICIO 4: Comprobad, de forma similar a los ejemplos anteriores, que el campo `CommentCount` de `Posts` es correcto, comprobándolo con la colección `Comments`. Definid los índices necesarios.

El resultado de la consulta de los números de comentarios que difieren debe ser vacío como en el ejemplo de los votos anterior.
